<center>
<h1> TP 4 : Programmation Dynamique </h1>
<h1> Année 2021-2022 - 2e année département Sciences du Numérique </h1>
<h1> ANACLET Paul </h1> 
<h1> BONREPAUX Rémi </h1>    
</center>

## Récupération des données
Cette fonction permet de **lire les données** du fichier passé en paramètre.

In [1]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

## Résolution du problème du sac à dos

L’algorithme ci-dessous permet de résoudre le problème du sac à dos en utilisant la **programmation dynamique**.
En effet, chaque étape correspond à un **sous-problème résolu optimalement**.
Les commentaires permettent d'expliquer plus en détail les différentes étapes de cet algorithme.

In [2]:
function solveSacADos(price, weight, capacity)
    nbObjet = length(price)
    C = zeros(nbObjet,capacity)
    solution_precedente =  Dict()
    solution_actuelle=  Dict()
    
# Initialisation la première ligne du tableau
    for j in 1:capacity
# Initialisation de la solution 
        solution_precedente[j] = []
        if j-weight[1] >= 0
            C[1,j] = price[1]
            push!(solution_precedente[j],1)
        end
    end
            
# Récurrence
    for i in 2:nbObjet      
        solution_actuelle = deepcopy(solution_precedente)  
        
        for j in 1:capacity
# Cas 1: l'objet peut rentrer dans le sac
            if j-weight[i] >= 0 
                if j-weight[i] == 0
# Cas 1.1: le poid de l'objet i est exactement la capacité maximale du sac
                    C[i,j] = max(C[i-1,j], price[i])
# Si on ne prend pas l'objet i, la solution ne change pas. Sinon on ne prend que l'objet i.
                    if max(C[i-1,j], price[i]) == price[i] 
                        solution_actuelle[j] = []
                        push!(solution_actuelle[j],i)
                    end
                else
# Cas 1.2: le poid de l'objet i est inférieur à la capacité maximale du sac
# Si on ne prend pas l'objet i, la solution ne change pas. Sinon on ajoute à la solution précedente l'objet i.
                    if max(C[i-1,j], C[i-1,j-weight[i]] + price[i]) == C[i-1,j-weight[i]] + price[i] 
                        solution_actuelle[j] = deepcopy(solution_precedente[j-weight[i]])
                        push!(solution_actuelle[j],i)
                    end
                    C[i,j] = max(C[i-1,j], C[i-1,j-weight[i]] + price[i])
                end
            else
# Cas 2: l'objet ne peut pas rentrer dans le sac
                C[i,j] = C[i-1,j]
            end
        end
        solution_precedente = deepcopy(solution_actuelle)
    end
    sol = C[nbObjet,capacity]
    
    return sol,solution_actuelle[capacity]
end

solveSacADos (generic function with 1 method)

Nous aurions pu remplacer le tableau C, de dimension (nbObjet,capacity) par deux vecteurs: un contenant les valeurs précedentes et un autre avec les valeurs actuelles. 
Cette amélioration **ne change rien à l'algorithme mais aurait permis d'économiser de la mémoire**.

# Analyse des résultats

Le code ci-dessous permet d'obtenir les **résultats** de l'algorithme pour **différentes données**. Il suffit de décommenter le nom des fichiers que nous voulons tester (**et de les récuperer au préalable sur moodle**). 

L'algorithme renvoi la liste sol où sont énumérés le **numéro des objets à prendre**. 
La solution s'en déduit directement en remplaçant les indices du vecteur sol par un 1 dans un vecteur nul ( de taille égale au nombre d'objet). 
Nous avons choisis de représenter la solution ainsi car ce vecteur est plus visuel, et il est plus facile de comparer les résultats avec le TP précedent. 

D'un coté, les **résultats sont cohérents** et l'algorithme est **plus simple à implanté** que celui du TP précedent. 
En revanche, cet algorithme nécessite en géneral **plus d'itérations** pour trouver la solution ce qui engendre un **temps d'éxecution est supérieur**. De plus, de nombreuses solutions existent pour optmiser la stratégie d'exploration des noeuds pour le BAB et nous avons vu que des solveurs performant n'avaient besoin que d'une ou deux itérations pour résoudre le problème du sac à dos. En revanche, il est **plus difficile d'optimiser cet algorithme dynamique**, puisque le nombre d'itération est fixé (en fonction de la capacité).


In [3]:
#Tests de performance sur les différents jeux de données
# Un fichier de chaque dossier
test = ["test.opb.txt","almost_strongly_correlated/knapPI_5_50_1000_1_-2096.opb.txt","circle/knapPI_16_20_1000_1_-2291.opb.txt" ,
"inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt","profit_ceiling/knapPI_15_20_1000_5_-969.opb.txt","similar_weights/knapPI_9_50_1000_1_-995.opb.txt",
"strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt","subset_sum/knapPI_6_50_1000_1_-994.opb.txt","uncorrelated/knapPI_1_50_10000_1_-68946.opb.txt"] 

for t in test
    price, weight, capacity = readKnaptxtInstance("instancesETU/KNAPnewformat/"*t)

    println(" _______________________     Test ",t,"    ______________________")
    @time max,sol = solveSacADos(price, weight, capacity)
   
    println("")
    println(" MAXIMUM = ",  max)
    println(" solution = ",sol)
    println("\n")
end



 _______________________     Test test.opb.txt    ______________________
  0.466096 seconds (235.74 k allocations: 13.616 MiB, 99.84% compilation time)

 MAXIMUM = 65.0
 solution = Any[2, 4]


 _______________________     Test almost_strongly_correlated/knapPI_5_50_1000_1_-2096.opb.txt    ______________________
  0.275306 seconds (816.14 k allocations: 45.548 MiB, 5.44% gc time, 6.59% compilation time)

 MAXIMUM = 2096.0
 solution = Any[7, 11, 14, 24, 26, 36, 37, 38, 39, 45, 49]


 _______________________     Test circle/knapPI_16_20_1000_1_-2291.opb.txt    ______________________
  0.076145 seconds (283.25 k allocations: 15.792 MiB, 15.23% gc time)

 MAXIMUM = 2291.0
 solution = Any[3, 6, 15, 17]


 _______________________     Test inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt    ______________________
  0.226400 seconds (736.37 k allocations: 38.708 MiB)

 MAXIMUM = 994.0
 solution = Any[38]


 _______________________     Test profit_ceiling/knapPI_15_20_1000_5_-969.opb.

# Résolution du problème du voyageur

Le code ci-dessous permet de résoudre le **problème du voyageur** en utilisant l’algorithme de **Bellman-Ford** vu en cours, permettant de calculer le plus court chemin entre un sommet source s et tous les autres sommets d’un graphe quelconque.

Les commentaires permettent d'expliquer en détail les différentes étapes du codes.

In [11]:
function bellman_ford(Mat,ptsDebut,ptsFinal)
# Initialisation vecteur solution 
    taille = size(Mat, 1)
    Sol = zeros(size(Mat, 1)) .+ Inf
    Sol[ptsDebut] = 0
# Chemin associé au vecteur solution
    Chemin = zeros(taille)
# Sommet à explorer
    explo = ptsDebut

# On s'arrete quand on a exploré toutes les solutions et qu'on arrive au point final
    while explo != ptsFinal 
# On itère sur tous les sommets accessibles depuis le sommet à explorer
        for i in 1:taille
            if i != explo
                minimum = min(Sol[explo] + Mat[explo,i], Sol[i])
                if minimum != Sol[i]
                    Chemin[i] = explo
                end
                Sol[i] = minimum
            end  
        end
# On change le sommet à explorer
        explo = (explo + 1)
    end

# Remonter pour connaitre le chemin
    Chemin = Int.(Chemin)
    explo = Chemin[ptsFinal] 
    CheminStr = string(ptsFinal)
    while explo != 0
        CheminStr = string(explo, " -> ",CheminStr )
        explo = Chemin[explo]
    end
    return Sol, CheminStr
end

bellman_ford (generic function with 1 method)

La cellule ci-dessous permet de **tester l'algorithme** de Bellman Ford avec les données du cours.
Les lettres sont remplacées par des nombres suivant l'ordre alphabétique.

In [14]:
# Matrice du problème du cours 
CoutChemin = [0 3 Inf Inf 5 Inf;Inf 0 4 Inf Inf Inf; Inf Inf 0 2 Inf Inf; Inf Inf Inf 0 Inf 3;Inf -1 Inf 9 0 Inf;Inf Inf Inf Inf Inf 0]

# A = 1, B = 2 ...
ptsDepart = 1
ptsFin = 6

Sol, CheminStr = bellman_ford(CoutChemin,ptsDepart,ptsFin)


println("Coût du chemin :")
println(Sol[ptsFin])
println("Chemin a prendre:")
println(CheminStr)

d2d3Coût du chemin :
7.0
Chemin a prendre:
1 -> 2 -> 3


### Amélioration possible:  

Afin d'améliorer la robustesse, nous **pourrions vérifier qu'il existe bien un chemin solution** qui mène au point final.
De plus, la boucle while, qui part du point de départ et itère jusqu'au point d'arrivé **nécessite que les points soient dans l'ordre croissant** et que le point d'arrivé est le plus grand. Cette contrainte peut-être satisafaite en pratique et permet de faciliter l'implantation de l'algorithme.